In [18]:
import numpy as np
import pandas as pd
import tensorflow_decision_forests as tfdf
from tensorflow_decision_forests.keras import core
from tensorflow_decision_forests.keras import core_inference
import keras_tuner as kt

# path = "../../../data/export-2-copy.csv"
# path = "../../../data/export-2-copy-2.csv"
# path = "../../../data/export-2-copy-2-copy.csv"
path = "../../../data/20230610-all-cant-understand-only.csv"
pandas_dataset = pd.read_csv(path)

# Display the first 3 examples.
# pd.set_option('display.max_columns', None)
# print(pandas_dataset.head(3))

label = "Tags"
is_false_trigger = "Is False Trigger"
# 0: AskRepeat
# 1: False Trigger
# 2: No Op

# classes = list(pandas_dataset[label].unique())
# print(f"Label classes: {classes}")
# >> Label classes: ['sub-TLF: Ask2Repeat', 'NoOp', 'False Trigger']
label_classes = []
for index, row in pandas_dataset.iterrows():
    if row[is_false_trigger] == True:
        label_classes.append(1)
    elif "sub-TLF: Ask2Repeat" in str(row[label]):
        label_classes.append(0)
    else:
        label_classes.append(2)

pandas_dataset[label] = label_classes

# for index, row in pandas_dataset.iterrows():
#     print(index, row[label], row[is_false_trigger] == True)

# pandas_dataset[label] = pandas_dataset[label].map(classes.index)

np.random.seed(1)
# Use the ~10% of the examples as the testing set
# and the remaining ~90% of the examples as the training set.
test_indices = np.random.rand(len(pandas_dataset)) < 0.1
pandas_train_dataset = pandas_dataset[~test_indices]
pandas_test_dataset = pandas_dataset[test_indices]

print("Training examples: ", len(pandas_train_dataset))
# print("Training examples: ", pandas_train_dataset)
# >> Training examples: 117

print("Testing examples: ", len(pandas_test_dataset))
# >> Testing examples: 17

tf_train_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(pandas_train_dataset, label=label)
# model = tfdf.keras.CartModel()
model = tfdf.keras.CartModel(features=[
    core.FeatureUsage(name="Post_itn_confidence", semantic=core_inference.FeatureSemantic.NUMERICAL),
    core.FeatureUsage(name="Signal_to_noise_ratio_decibels", semantic=core_inference.FeatureSemantic.NUMERICAL),
    core.FeatureUsage(name="cat_category", semantic=core_inference.FeatureSemantic.CATEGORICAL)
], exclude_non_specified_features=True, min_examples=100, validation_ratio=0.0)

model.fit(tf_train_dataset)

model.compile("accuracy")
print("Train evaluation: ", model.evaluate(tf_train_dataset, return_dict=True))
# >> Train evaluation:  {'loss': 0.0, 'accuracy': 0.96116}

tf_test_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(pandas_test_dataset, label=label)
print("Test evaluation: ", model.evaluate(tf_test_dataset, return_dict=True))
# >> Test evaluation:  {'loss': 0.0, 'accuracy': 0.97142}
tfdf.model_plotter.plot_model_in_colab(model, max_depth=10)







# def build_model(hp):
#   model = tfdf.keras.CartModel(
#       min_examples=hp.Choice("min_examples",
#           # Try four possible values for "min_examples" hyperparameter.
#           # min_examples=10 would limit the growth of the decision tree,
#           # while min_examples=1 would lead to deeper decision trees.
#          [1, 2, 5, 10]),
#       validation_ratio=hp.Choice("validation_ratio",
#          # Three possible values for the "validation_ratio" hyperparameter.
#          [0.0, 0.05, 0.10]),
#       )
#   model.compile("accuracy")
#   return model
# 
# tuner = kt.RandomSearch(
#     build_model,
#     objective="val_accuracy",
#     max_trials=10,
#     directory="../../../data/tmp/tuner",
#     project_name="tune_cart")
# 
# tuner.search(x=tf_train_dataset, validation_data=tf_test_dataset)
# best_model = tuner.get_best_models()[0]
# 
# print("Best hyperparameters: ", tuner.get_best_hyperparameters()[0].values)
# # >> Best hyperparameters:  {'min_examples': 5, 'validation_ratio': 0.0}
# 
# model = tfdf.keras.CartModel(min_examples=5, validation_ratio=0.0)
# model.fit(tf_train_dataset)
# 
# model.compile("accuracy")
# print("Test evaluation: ", model.evaluate(tf_test_dataset, return_dict=True))
# >> Test evaluation:  {'loss': 0.0, 'accuracy': 1.0}
# 
# tfdf.model_plotter.plot_model_in_colab(model, max_depth=10)


Training examples:  903
Testing examples:  97


Use /var/folders/kv/brh7vhc932l_vjg21r3q83wc0000gn/T/tmpz6utieqb as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.116132. Found 903 examples.
Training model...
Model trained in 0:00:00.007830
Compiling model...


[INFO 24-01-23 18:06:15.8978 PST kernel.cc:1233] Loading model from path /var/folders/kv/brh7vhc932l_vjg21r3q83wc0000gn/T/tmpz6utieqb/model/ with prefix e70d0e374b314b0d
[INFO 24-01-23 18:06:15.8980 PST decision_forest.cc:660] Model loaded with 1 root(s), 13 node(s), and 3 input feature(s).
[INFO 24-01-23 18:06:15.8980 PST abstract_model.cc:1344] Engine "RandomForestGeneric" built
[INFO 24-01-23 18:06:15.8980 PST kernel.cc:1061] Use fast generic engine


Model compiled.
1/1 [==============================] - 0s 99ms/step - loss: 0.0000e+00 - accuracy: 0.8560
Train evaluation:  {'loss': 0.0, 'accuracy': 0.8560354113578796}


1/1 [==============================] - 0s 88ms/step - loss: 0.0000e+00 - accuracy: 0.8144
Test evaluation:  {'loss': 0.0, 'accuracy': 0.8144329786300659}
